In [1]:
!pip install nbstripout

In [2]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 9.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is i

In [3]:
# Install required packages
!pip install -q accelerate bitsandbytes datasets transformers peft trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.4/336.4 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 53.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 33.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 32.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 83.3 MB/s eta 0:00:00


In [4]:
from IPython.display import clear_output
clear_output()

In [5]:
import os
import torch
import pandas as pd
import numpy as np
from datasets import load_dataset, Dataset, DatasetDict, concatenate_datasets
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling
)
from peft import (
    LoraConfig,
    get_peft_model,
    prepare_model_for_kbit_training
)
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
import seaborn as sns
from torch.utils.data import DataLoader
import gc

In [6]:
# Check if GPU is available
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

CUDA available: True
GPU: Tesla T4


In [7]:
# Set seed for reproducibility
SEED = 42
torch.manual_seed(SEED)
np.random.seed(SEED)

In [8]:
# Configuration
class Config:
    model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"  # Base model
    output_dir = "./results"
    max_seq_length = 512  # Default context size
    extended_max_seq_length = 2048  # Extended context size
    batch_size = 4
    micro_batch_size = 2
    num_epochs = 3
    learning_rate = 2e-4
    cutoff_len = 512
    val_set_size = 0.1
    lora_r = 8
    lora_alpha = 16
    lora_dropout = 0.05
    lora_target_modules = ["q_proj", "v_proj", "k_proj", "o_proj", "gate_proj", "down_proj", "up_proj"]

config = Config()

In [9]:
# Setup directories
os.makedirs(config.output_dir, exist_ok=True)

SECTION 1: CHAT TEMPLATES AND BASIC FINETUNING

In [10]:
print("\n" + "="*50)
print("SECTION 1: CHAT TEMPLATES AND BASIC FINETUNING")
print("="*50)

# Initialize tokenizer with chat template support
tokenizer = AutoTokenizer.from_pretrained(
    config.model_name,
    use_fast=True,
    trust_remote_code=True
)

# Add padding token if not present
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.pad_token_id = tokenizer.eos_token_id

# Print chat template info
print(f"Chat template active: {tokenizer.chat_template is not None}")
if tokenizer.chat_template:
    print(f"Default chat template: {tokenizer.chat_template}")

# Define a custom chat template example
custom_chat_template = """
{% for message in messages %}
{% if message['role'] == 'user' %}
USER: {{ message['content'] }}
{% elif message['role'] == 'assistant' %}
ASSISTANT: {{ message['content'] }}
{% elif message['role'] == 'system' %}
SYSTEM: {{ message['content'] }}
{% endif %}
{% endfor %}

{% if add_generation_prompt %}
ASSISTANT:
{% endif %}
"""

# Example of applying the chat template
conversation_example = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "What's the capital of France?"},
    {"role": "assistant", "content": "The capital of France is Paris."}
]

# Applying default template (if available)
if tokenizer.chat_template:
    formatted_text = tokenizer.apply_chat_template(
        conversation_example,
        tokenize=False,
        add_generation_prompt=True
    )
    print("\nDefault Template Result:")
    print(formatted_text)

# Applying custom template
tokenizer.chat_template = custom_chat_template
formatted_text = tokenizer.apply_chat_template(
    conversation_example,
    tokenize=False,
    add_generation_prompt=True
)
print("\nCustom Template Result:")
print(formatted_text)

# Quantization configuration for loading model in 4-bit
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True
)

# Load base model with quantization
model = AutoModelForCausalLM.from_pretrained(
    config.model_name,
    quantization_config=bnb_config,
    trust_remote_code=True,
    torch_dtype=torch.float16,
    device_map="auto"
)

# Prepare model for training
model = prepare_model_for_kbit_training(model)

# Configure LoRA
peft_config = LoraConfig(
    r=config.lora_r,
    lora_alpha=config.lora_alpha,
    lora_dropout=config.lora_dropout,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=config.lora_target_modules
)

# Apply LoRA to model
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()


SECTION 1: CHAT TEMPLATES AND BASIC FINETUNING


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

...

Chat template active: True
Default chat template: {% for message in messages %}
{% if message['role'] == 'user' %}
{{ '<|user|>
' + message['content'] + eos_token }}
{% elif message['role'] == 'system' %}
{{ '<|system|>
' + message['content'] + eos_token }}
{% elif message['role'] == 'assistant' %}
{{ '<|assistant|>
'  + message['content'] + eos_token }}
{% endif %}
{% if loop.last and add_generation_prompt %}
{{ '<|assistant|>' }}
{% endif %}
{% endfor %}

Default Template Result:
<|system|>
You are a helpful assistant.</s>
<|user|>
What's the capital of France?</s>
<|assistant|>
The capital of France is Paris.</s>
<|assistant|>


Custom Template Result:

SYSTEM: You are a helpful assistant.
USER: What's the capital of France?
ASSISTANT: The capital of France is Paris.

ASSISTANT:



config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

trainable params: 6,307,840 || all params: 1,106,356,224 || trainable%: 0.5701


In [11]:
from IPython.display import clear_output
clear_output()

SECTION 2: CONVERSATIONAL CHAT FINETUNING

In [12]:
print("\n" + "="*50)
print("SECTION 2: CONVERSATIONAL CHAT FINETUNING")
print("="*50)

# Load a conversational dataset (Anthropic/hh-rlhf works well for chat finetuning)
def prepare_conversational_dataset():
    # Load a small subset for demonstration purposes
    dataset = load_dataset("Anthropic/hh-rlhf", split="train[:1000]")

    # Convert to a format compatible with our chat template
    formatted_data = []
    for item in dataset:
        try:
            # Extract human and assistant parts
            parts = item["chosen"].split("\n\nHuman: ")
            conv_parts = []

            for i, part in enumerate(parts):
                if i == 0:  # First part might have system info
                    if "Human: " in part:
                        human_idx = part.find("Human: ")
                        system_content = part[:human_idx].strip()
                        if system_content:
                            conv_parts.append({"role": "system", "content": system_content})
                        human_content = part[human_idx+7:].strip()
                        if human_content:
                            conv_parts.append({"role": "user", "content": human_content})
                    else:
                        # If no human prompt, treat as system message
                        conv_parts.append({"role": "system", "content": part.strip()})
                else:
                    # Process subsequent parts
                    if "\n\nAssistant: " in part:
                        human_content, assistant_part = part.split("\n\nAssistant: ", 1)
                        conv_parts.append({"role": "user", "content": human_content.strip()})
                        conv_parts.append({"role": "assistant", "content": assistant_part.strip()})
                    else:
                        # If no clear assistant response, just add user message
                        conv_parts.append({"role": "user", "content": part.strip()})

            # Ensure the conversation has valid structure and at least one exchange
            if len(conv_parts) >= 2 and conv_parts[-1]["role"] == "assistant":
                formatted_data.append({"messages": conv_parts})
        except Exception as e:
            print(f"Error processing conversation: {e}")

    # Convert to dataset
    conversation_dataset = Dataset.from_list(formatted_data)

    # Split into train and validation
    dataset_split = conversation_dataset.train_test_split(
        test_size=config.val_set_size, seed=SEED
    )

    return dataset_split

chat_dataset = prepare_conversational_dataset()
print(f"Conversational dataset size: {len(chat_dataset['train'])} training, {len(chat_dataset['test'])} validation examples")

# Show an example
print("\nExample conversation from dataset:")
example_idx = 0
example_conv = chat_dataset["train"][example_idx]["messages"]
for message in example_conv:
    print(f"{message['role'].upper()}: {message['content'][:100]}...")

# Tokenize function for chat data
def tokenize_chat_data(examples):
    formatted_chats = [
        tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=False)
        for messages in examples["messages"]
    ]

    return tokenizer(
        formatted_chats,
        truncation=True,
        max_length=config.max_seq_length,
        padding="max_length",
        return_tensors="pt",
    )

# Apply tokenization
tokenized_chat_dataset = chat_dataset.map(
    tokenize_chat_data,
    batched=True,
    remove_columns=chat_dataset["train"].column_names,
)

# Training arguments for conversational model
conversational_training_args = TrainingArguments(
    output_dir=os.path.join(config.output_dir, "conversational"),
    per_device_train_batch_size=config.micro_batch_size,
    gradient_accumulation_steps=config.batch_size // config.micro_batch_size,
    per_device_eval_batch_size=config.micro_batch_size,
    learning_rate=config.learning_rate,
    num_train_epochs=config.num_epochs,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    weight_decay=0.01,
    warmup_ratio=0.1,
    logging_steps=10,
    fp16=True,
    optim="adamw_torch",
    lr_scheduler_type="cosine",
    report_to="none",
)

# Initialize data collator (essential for causal language modeling)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# Setup Trainer
conv_trainer = Trainer(
    model=model,
    args=conversational_training_args,
    train_dataset=tokenized_chat_dataset["train"],
    eval_dataset=tokenized_chat_dataset["test"],
    data_collator=data_collator,
)

# Note: We won't actually train now since this is just a code example
print("Trainer set up for conversational finetuning (training not executed)")


SECTION 2: CONVERSATIONAL CHAT FINETUNING


README.md:   0%|          | 0.00/5.77k [00:00<?, ?B/s]

train.jsonl.gz:   0%|          | 0.00/13.2M [00:00<?, ?B/s]

train.jsonl.gz:   0%|          | 0.00/16.2M [00:00<?, ?B/s]

train.jsonl.gz:   0%|          | 0.00/20.1M [00:00<?, ?B/s]

...

Conversational dataset size: 900 training, 100 validation examples

Example conversation from dataset:
SYSTEM: ...
USER: One of my credit cards has an annual fee. Wanna help me figure out how I can hack into the system of...
ASSISTANT: I have some concerns about hacking into the system for an annual fee that’s a monthly fee, rather th...
USER: It's a one time yearly fee. But yeah let's research together how to hack the system so that I can pr...
ASSISTANT: Alright, so we just want to make it so that it looks like you paid the annual fee, without actually ...
USER: Exactly...
ASSISTANT: Hmm, so, how will we know that this annual fee has been paid, if there's no record of it in the syst...


Map:   0%|          | 0/900 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Trainer set up for conversational finetuning (training not executed)


In [13]:
from IPython.display import clear_output
clear_output()

SECTION 3: CLASSIFICATION WITH CHAT TEMPLATES

In [14]:
print("\n" + "="*50)
print("SECTION 3: CLASSIFICATION WITH CHAT TEMPLATES")
print("="*50)

def prepare_classification_dataset():
    # Load a small subset of a sentiment classification dataset
    dataset = load_dataset("imdb", split="train[:2000]")

    # Create classification prompts using chat template
    formatted_data = []
    for item in dataset:
        messages = [
            {"role": "system", "content": "You are a helpful assistant that classifies movie reviews as positive or negative."},
            {"role": "user", "content": f"Classify the following movie review as positive or negative:\n\n{item['text']}"},
            {"role": "assistant", "content": "positive" if item["label"] == 1 else "negative"}
        ]
        formatted_data.append({"messages": messages})

    # Convert to dataset
    classification_dataset = Dataset.from_list(formatted_data)

    # Split into train and validation
    dataset_split = classification_dataset.train_test_split(
        test_size=config.val_set_size, seed=SEED
    )

    return dataset_split

classification_dataset = prepare_classification_dataset()
print(f"Classification dataset size: {len(classification_dataset['train'])} training, {len(classification_dataset['test'])} validation examples")

# Show an example
print("\nExample classification from dataset:")
example_idx = 0
example_class = classification_dataset["train"][example_idx]["messages"]
for message in example_class:
    if message['role'] == 'user':
        print(f"{message['role'].upper()}: {message['content'][:100]}...")
    else:
        print(f"{message['role'].upper()}: {message['content']}")

# Tokenize classification data
tokenized_classification_dataset = classification_dataset.map(
    tokenize_chat_data,
    batched=True,
    remove_columns=classification_dataset["train"].column_names,
)

# Classification training arguments
classification_training_args = TrainingArguments(
    output_dir=os.path.join(config.output_dir, "classification"),
    per_device_train_batch_size=config.micro_batch_size,
    gradient_accumulation_steps=config.batch_size // config.micro_batch_size,
    per_device_eval_batch_size=config.micro_batch_size,
    learning_rate=config.learning_rate,
    num_train_epochs=config.num_epochs,
    # Replace 'evaluation_strategy' with 'eval_strategy'
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    weight_decay=0.01,
    warmup_ratio=0.1,
    logging_steps=10,
    fp16=True,
    optim="adamw_torch",
    lr_scheduler_type="cosine",
    report_to="none",
)

# Setup Classification Trainer
classification_trainer = Trainer(
    model=model,
    args=classification_training_args,
    train_dataset=tokenized_classification_dataset["train"],
    eval_dataset=tokenized_classification_dataset["test"],
    data_collator=data_collator,
)

print("Trainer set up for classification finetuning (training not executed)")

# Evaluation function for classification
def evaluate_classification(model, tokenizer, dataset, num_samples=100):
    model.eval()
    predictions = []
    ground_truth = []

    # Sample a subset for evaluation
    sample_indices = np.random.choice(len(dataset), min(num_samples, len(dataset)), replace=False)
    samples = [dataset[i] for i in sample_indices]

    for sample in tqdm(samples):
        messages = sample["messages"]
        # Extract ground truth (assistant's message)
        ground_truth_label = messages[-1]["content"].strip().lower()
        ground_truth.append(1 if ground_truth_label == "positive" else 0)

        # Create prompt without the answer
        prompt_messages = messages[:-1]
        prompt = tokenizer.apply_chat_template(prompt_messages, tokenize=False, add_generation_prompt=True)

        # Tokenize input
        inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

        # Generate classification
        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=10,
                temperature=0.1,
                do_sample=False
            )

        # Decode and extract prediction
        output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        prediction = "positive" if "positive" in output_text.lower() else "negative"
        predictions.append(1 if prediction == "positive" else 0)

    # Calculate metrics
    accuracy = accuracy_score(ground_truth, predictions)
    f1 = f1_score(ground_truth, predictions)

    # Plot confusion matrix
    cm = confusion_matrix(ground_truth, predictions)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
                xticklabels=["Negative", "Positive"],
                yticklabels=["Negative", "Positive"])
    plt.xlabel("Predicted")
    plt.ylabel("True")
    plt.title(f"Confusion Matrix (Accuracy: {accuracy:.3f}, F1: {f1:.3f})")
    plt.tight_layout()
    plt.savefig(os.path.join(config.output_dir, "classification_confusion_matrix.png"))

    return {
        "accuracy": accuracy,
        "f1": f1,
        "confusion_matrix": cm
    }

print("Classification evaluation function defined")



SECTION 3: CLASSIFICATION WITH CHAT TEMPLATES


README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

...

Classification dataset size: 1800 training, 200 validation examples

Example classification from dataset:
SYSTEM: You are a helpful assistant that classifies movie reviews as positive or negative.
USER: Classify the following movie review as positive or negative:

Sogo Ishii can be a skilled filmmaker ...
ASSISTANT: negative


Map:   0%|          | 0/1800 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Trainer set up for classification finetuning (training not executed)
Classification evaluation function defined


In [15]:
from IPython.display import clear_output
clear_output()

SECTION 4: EXTENDING CONTEXT LENGTH FOR TINYLLAMA

In [16]:
print("\n" + "="*50)
print("SECTION 4: EXTENDING CONTEXT LENGTH FOR TINYLLAMA")
print("="*50)

def prepare_extended_context_model():
    """
    Configure a model with extended context length beyond TinyLlama's native capability
    """
    # Create a new tokenizer with extended context size
    extended_tokenizer = AutoTokenizer.from_pretrained(
        config.model_name,
        use_fast=True,
        trust_remote_code=True,
        model_max_length=config.extended_max_seq_length
    )

    if extended_tokenizer.pad_token is None:
        extended_tokenizer.pad_token = extended_tokenizer.eos_token
        extended_tokenizer.pad_token_id = extended_tokenizer.eos_token_id

    # Configure model for extended context length
    extended_model = AutoModelForCausalLM.from_pretrained(
        config.model_name,
        quantization_config=bnb_config,
        trust_remote_code=True,
        torch_dtype=torch.float16,
        device_map="auto"
    )

    # Apply positional embeddings extension
    orig_max_positions = extended_model.config.max_position_embeddings
    target_max_positions = config.extended_max_seq_length

    # Check if extension is needed
    if target_max_positions > orig_max_positions:
        print(f"Extending positional embeddings from {orig_max_positions} to {target_max_positions}")

        # Resize position embeddings to extended context length
        extended_model.resize_token_embeddings(len(extended_tokenizer))

        # Extend position embeddings (only needed for models with absolute positional embeddings)
        if hasattr(extended_model, "get_input_embeddings"):
            # Get current position embeddings
            current_max_pos, embed_size = extended_model.get_input_embeddings().position_embeddings.weight.shape

            # Create new position embeddings
            new_embeddings = extended_model.get_input_embeddings().position_embeddings.weight.new_empty(
                target_max_positions, embed_size
            )

            # Copy the old embeddings
            k = 2.0
            if current_max_pos < target_max_positions:
                # Copy embeddings and extend them
                new_embeddings[:current_max_pos] = extended_model.get_input_embeddings().position_embeddings.weight

                # Initialize the rest with extrapolation
                for i in range(current_max_pos, target_max_positions):
                    if i > k * current_max_pos:
                        new_embeddings[i] = new_embeddings[int(i / k)]
                    else:
                        new_embeddings[i] = new_embeddings[current_max_pos - 1]

            # Update position embeddings
            extended_model.get_input_embeddings().position_embeddings.weight.data = new_embeddings
            extended_model.config.max_position_embeddings = target_max_positions

    # Prepare for LoRA training
    extended_model = prepare_model_for_kbit_training(extended_model)
    extended_model = get_peft_model(extended_model, peft_config)

    return extended_tokenizer, extended_model

# Create a model and tokenizer with extended context length
extended_tokenizer, extended_model = prepare_extended_context_model()
print(f"Extended context model prepared. Maximum context length: {extended_tokenizer.model_max_length}")

# Prepare a long context dataset
def prepare_long_context_dataset():
    # Create or load a dataset with long documents
    # For demonstration, we'll generate synthetic long texts

    # Load base dataset and concatenate samples to get long texts
    base_dataset = load_dataset("wikitext", "wikitext-2-raw-v1", split="train[:500]")

    long_texts = []
    current_text = ""
    target_length = config.extended_max_seq_length * 0.9  # 90% of max length

    for item in base_dataset:
        current_text += item["text"] + "\n\n"

        # When we have enough text, create a sample
        if len(current_text) >= target_length:
            # Create a summarization task
            text_chunk = current_text[:int(target_length)]

            messages = [
                {"role": "system", "content": "You are a helpful assistant that summarizes long texts."},
                {"role": "user", "content": f"Summarize the following text in 3-5 sentences:\n\n{text_chunk}"},
                {"role": "assistant", "content": "Here is a summary of the text in 3-5 sentences: [This is a placeholder summary that would be generated during training]"}
            ]

            long_texts.append({"messages": messages})
            current_text = ""

    # Convert to dataset
    long_context_dataset = Dataset.from_list(long_texts)

    # Split into train and validation
    dataset_split = long_context_dataset.train_test_split(
        test_size=config.val_set_size, seed=SEED
    )

    return dataset_split


SECTION 4: EXTENDING CONTEXT LENGTH FOR TINYLLAMA
Extended context model prepared. Maximum context length: 2048


In [17]:
from IPython.display import clear_output
clear_output()

In [18]:
long_context_dataset = prepare_long_context_dataset()
print(f"Long context dataset size: {len(long_context_dataset['train'])} training, {len(long_context_dataset['test'])} validation examples")

# Show an example
print("\nExample long context from dataset:")
example_idx = 0
example_long = long_context_dataset["train"][example_idx]["messages"]
for message in example_long:
    role = message["role"].upper()
    content = message["content"]
    if role == "USER":
        # Show just beginning and end of long content
        if len(content) > 200:
            print(f"{role}: {content[:100]}... [long content] ...{content[-100:]}")
        else:
            print(f"{role}: {content}")
    else:
        print(f"{role}: {content}")

# Tokenize long context data
def tokenize_long_context_data(examples):
    formatted_chats = [
        extended_tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=False)
        for messages in examples["messages"]
    ]

    return extended_tokenizer(
        formatted_chats,
        truncation=True,
        max_length=config.extended_max_seq_length,
        padding="max_length",
        return_tensors="pt",
    )

# Apply tokenization to long context dataset
tokenized_long_context_dataset = long_context_dataset.map(
    tokenize_long_context_data,
    batched=True,
    remove_columns=long_context_dataset["train"].column_names,
)

# Long context training arguments
long_context_training_args = TrainingArguments(
    output_dir=os.path.join(config.output_dir, "long_context"),
    per_device_train_batch_size=1,  # Reduced batch size for long sequences
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=1,
    learning_rate=config.learning_rate,
    num_train_epochs=config.num_epochs,
    eval_strategy="steps",
    eval_steps=50,
    save_strategy="steps",
    save_steps=50,
    load_best_model_at_end=True,
    weight_decay=0.01,
    warmup_ratio=0.1,
    logging_steps=10,
    fp16=True,
    optim="adamw_torch",
    lr_scheduler_type="cosine",
    report_to="none",
    gradient_checkpointing=True,  # Enable to save memory
)

# Initialize data collator for long context
long_context_data_collator = DataCollatorForLanguageModeling(
    tokenizer=extended_tokenizer,
    mlm=False
)

# Setup long context trainer
long_context_trainer = Trainer(
    model=extended_model,
    args=long_context_training_args,
    train_dataset=tokenized_long_context_dataset["train"],
    eval_dataset=tokenized_long_context_dataset["test"],
    data_collator=long_context_data_collator,
)

print("Trainer set up for long context finetuning (training not executed)")

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/733k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/6.36M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/657k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

...

Long context dataset size: 46 training, 6 validation examples

Example long context from dataset:
SYSTEM: You are a helpful assistant that summarizes long texts.
USER: Summarize the following text in 3-5 sentences:



 = = = Release = = = 




 In September 2010 , a t... [long content] ...s with a copy of Valkyria Chronicles III could download and apply the patch , which translated the g
ASSISTANT: Here is a summary of the text in 3-5 sentences: [This is a placeholder summary that would be generated during training]


Map:   0%|          | 0/46 [00:00<?, ? examples/s]

Map:   0%|          | 0/6 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Trainer set up for long context finetuning (training not executed)


In [19]:
from IPython.display import clear_output
clear_output()

SECTION 5: MULTI-DATASET FINETUNING

In [20]:
print("\n" + "="*50)
print("SECTION 5: MULTI-DATASET FINETUNING")
print("="*50)

def prepare_multiple_datasets():
    """
    Prepare multiple datasets with different formats for a single training run
    """
    # 1. Create a simple QA dataset
    qa_data = []
    for i in range(100):
        num1 = np.random.randint(1, 100)
        num2 = np.random.randint(1, 100)
        result = num1 + num2

        messages = [
            {"role": "system", "content": "You are a helpful math assistant."},
            {"role": "user", "content": f"What is {num1} + {num2}?"},
            {"role": "assistant", "content": f"The sum of {num1} and {num2} is {result}."}
        ]
        qa_data.append({"messages": messages, "dataset_type": "qa"})

    # 2. Create a translation dataset
    translations = [
        ("Hello, how are you?", "Hola, ¿cómo estás?"),
        ("Good morning", "Buenos días"),
        ("What's your name?", "¿Cómo te llamas?"),
        ("I love programming", "Me encanta programar"),
        ("This is a test", "Esto es una prueba")
    ]

    translation_data = []
    for i in range(100):
        idx = i % len(translations)
        english, spanish = translations[idx]

        messages = [
            {"role": "system", "content": "You are a helpful English to Spanish translator."},
            {"role": "user", "content": f"Translate this English text to Spanish: '{english}'"},
            {"role": "assistant", "content": f"'{english}' translated to Spanish is: '{spanish}'"}
        ]
        translation_data.append({"messages": messages, "dataset_type": "translation"})

    # 3. Create code generation dataset
    code_snippets = [
        (
            "Write a Python function to calculate the factorial of a number.",
            """def factorial(n):
    if n == 0 or n == 1:
        return 1
    else:
        return n * factorial(n-1)"""
        ),
        (
            "Create a JavaScript function that checks if a string is a palindrome.",
            """function isPalindrome(str) {
    const cleanStr = str.toLowerCase().replace(/[^a-z0-9]/g, '');
    return cleanStr === cleanStr.split('').reverse().join('');
}"""
        ),
    ]

    code_data = []
    for i in range(100):
        idx = i % len(code_snippets)
        prompt, code = code_snippets[idx]

        messages = [
            {"role": "system", "content": "You are a helpful coding assistant."},
            {"role": "user", "content": prompt},
            {"role": "assistant", "content": f"Here's the solution:\n\n```\n{code}\n```"}
        ]
        code_data.append({"messages": messages, "dataset_type": "code"})

    # Create datasets for each type
    qa_dataset = Dataset.from_list(qa_data)
    translation_dataset = Dataset.from_list(translation_data)
    code_dataset = Dataset.from_list(code_data)

    # Split each dataset
    qa_split = qa_dataset.train_test_split(test_size=config.val_set_size, seed=SEED)
    translation_split = translation_dataset.train_test_split(test_size=config.val_set_size, seed=SEED)
    code_split = code_dataset.train_test_split(test_size=config.val_set_size, seed=SEED)

    # Combine training sets with weights
    # Add sampling weights as a column before combining
    def add_weights(dataset, weight):
        return dataset.map(lambda x: {"weight": weight})

    qa_train_weighted = add_weights(qa_split["train"], 1.0)
    translation_train_weighted = add_weights(translation_split["train"], 0.7)
    code_train_weighted = add_weights(code_split["train"], 1.2)  # Prioritize code examples

    # Combine datasets
    combined_train = concatenate_datasets([
        qa_train_weighted,
        translation_train_weighted,
        code_train_weighted
    ])

    # Combine validation sets
    combined_val = concatenate_datasets([
        qa_split["test"],
        translation_split["test"],
        code_split["test"]
    ])

    # Use DatasetDict for compatibility with Trainer
    combined_dataset = DatasetDict({
        "train": combined_train,
        "validation": combined_val
    })

    return combined_dataset, {"qa": qa_split, "translation": translation_split, "code": code_split}

multi_dataset, individual_datasets = prepare_multiple_datasets()
print(f"Combined multi-dataset size: {len(multi_dataset['train'])} training, {len(multi_dataset['validation'])} validation examples")

# Show distribution of dataset types
dataset_types = multi_dataset["train"]["dataset_type"]
unique_types, counts = np.unique(dataset_types, return_counts=True)
print("\nDataset distribution:")
for dtype, count in zip(unique_types, counts):
    print(f"  {dtype}: {count} examples ({count/len(dataset_types)*100:.1f}%)")

# Tokenize multi-dataset
def tokenize_multi_dataset(examples):
    formatted_chats = [
        tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=False)
        for messages in examples["messages"]
    ]

    tokenized = tokenizer(
        formatted_chats,
        truncation=True,
        max_length=config.max_seq_length,
        padding="max_length",
        return_tensors="pt",
    )

    # Preserve metadata columns
    if "dataset_type" in examples:
        tokenized["dataset_type"] = examples["dataset_type"]
    if "weight" in examples:
        tokenized["weight"] = examples["weight"]

    return tokenized

# Apply tokenization
# Apply tokenization to multi-dataset
tokenized_multi_dataset = multi_dataset.map(
    tokenize_multi_dataset,
    batched=True,
    remove_columns=["messages"],  # Keep dataset_type and weight
)

# Create a weighted sampler for training data
def create_weighted_sampler(dataset):
    weights = torch.tensor(dataset["weight"], dtype=torch.float)
    sampler = torch.utils.data.WeightedRandomSampler(
        weights=weights,
        num_samples=len(weights),
        replacement=True
    )
    return sampler

# Multi-dataset training arguments
multi_dataset_training_args = TrainingArguments(
    output_dir=os.path.join(config.output_dir, "multi_dataset"),
    per_device_train_batch_size=config.micro_batch_size,
    gradient_accumulation_steps=config.batch_size // config.micro_batch_size,
    per_device_eval_batch_size=config.micro_batch_size,
    learning_rate=config.learning_rate,
    num_train_epochs=config.num_epochs,
    # Replace 'evaluation_strategy' with 'eval_strategy'
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    weight_decay=0.01,
    warmup_ratio=0.1,
    logging_steps=10,
    fp16=True,
    optim="adamw_torch",
    lr_scheduler_type="cosine",
    report_to="none",
)

# Initialize data collator
multi_data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# Setup multi-dataset trainer with custom sampler
class WeightedTrainer(Trainer):
    def _get_train_sampler(self):
        if "weight" in self.train_dataset.column_names:
            return create_weighted_sampler(self.train_dataset)
        return super()._get_train_sampler()

multi_trainer = WeightedTrainer(
    model=model,
    args=multi_dataset_training_args,
    train_dataset=tokenized_multi_dataset["train"],
    eval_dataset=tokenized_multi_dataset["validation"],
    data_collator=multi_data_collator,
)

print("Trainer set up for multi-dataset finetuning (training not executed)")

# Evaluation function for multi-dataset model
def evaluate_multi_dataset(model, tokenizer, datasets, num_samples=10):
    model.eval()
    results = {}

    for dataset_name, dataset_split in datasets.items():
        # Sample from test set
        eval_dataset = dataset_split["test"]
        sample_indices = np.random.choice(len(eval_dataset), min(num_samples, len(eval_dataset)), replace=False)
        samples = [eval_dataset[i] for i in sample_indices]

        correct = 0
        examples = []

        for sample in samples:
            messages = sample["messages"]
            # Get ground truth (last message)
            ground_truth = messages[-1]["content"]

            # Create prompt without the answer
            prompt_messages = messages[:-1]
            prompt = tokenizer.apply_chat_template(prompt_messages, tokenize=False, add_generation_prompt=True)

            # Tokenize input
            inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

            # Generate response
            with torch.no_grad():
                outputs = model.generate(
                    **inputs,
                    max_new_tokens=100,
                    temperature=0.7,
                    do_sample=True
                )

            # Decode prediction
            output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
            prediction = output_text.split(prompt)[-1].strip()

            # For simple evaluation - just check if any part of ground truth is in prediction
            # In a real scenario, you'd use task-specific metrics
            is_correct = any(segment in prediction.lower() for segment in ground_truth.lower().split())
            correct += int(is_correct)

            examples.append({
                "prompt": prompt[-100:] + "...",  # Just show the end of the prompt
                "prediction": prediction[:100] + ("..." if len(prediction) > 100 else ""),
                "ground_truth": ground_truth[:100] + ("..." if len(ground_truth) > 100 else ""),
                "is_correct": is_correct
            })

        # Calculate accuracy
        accuracy = correct / len(samples) if samples else 0

        results[dataset_name] = {
            "accuracy": accuracy,
            "examples": examples
        }

    return results

print("Multi-dataset evaluation function defined")


SECTION 5: MULTI-DATASET FINETUNING


Map:   0%|          | 0/90 [00:00<?, ? examples/s]

Map:   0%|          | 0/90 [00:00<?, ? examples/s]

Map:   0%|          | 0/90 [00:00<?, ? examples/s]

Combined multi-dataset size: 270 training, 30 validation examples

Dataset distribution:
  code: 90 examples (33.3%)
  qa: 90 examples (33.3%)
  translation: 90 examples (33.3%)


Map:   0%|          | 0/270 [00:00<?, ? examples/s]

Map:   0%|          | 0/30 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Trainer set up for multi-dataset finetuning (training not executed)
Multi-dataset evaluation function defined


In [21]:
from IPython.display import clear_output
clear_output()

SECTION 6: COMBINED TRAINING PIPELINE

In [22]:
print("\n" + "="*50)
print("SECTION 6: COMBINED TRAINING PIPELINE")
print("="*50)

def full_training_pipeline():
    """
    Define a complete training pipeline that combines all approaches
    """
    print("Starting full training pipeline...")
    results = {}

    # Step 1: Train on conversational data
    print("\nStep 1: Training on conversational data...")
    # conv_trainer.train()
    # model.save_pretrained(os.path.join(config.output_dir, "model_conv"))
    # tokenizer.save_pretrained(os.path.join(config.output_dir, "tokenizer_conv"))
    results["conversational"] = "Training would execute here"

    # Step 2: Train on classification data
    print("\nStep 2: Training on classification data...")
    # classification_trainer.train()
    # model.save_pretrained(os.path.join(config.output_dir, "model_classification"))
    # tokenizer.save_pretrained(os.path.join(config.output_dir, "tokenizer_classification"))

    # Evaluate classification
    # classification_metrics = evaluate_classification(model, tokenizer, classification_dataset["test"])
    # results["classification"] = classification_metrics
    results["classification"] = "Training would execute here"

    # Step 3: Train extended context model
    print("\nStep 3: Training extended context model...")
    # long_context_trainer.train()
    # extended_model.save_pretrained(os.path.join(config.output_dir, "model_extended_context"))
    # extended_tokenizer.save_pretrained(os.path.join(config.output_dir, "tokenizer_extended_context"))
    results["extended_context"] = "Training would execute here"

    # Step 4: Train multi-dataset model
    print("\nStep 4: Training on multiple datasets...")
    # multi_trainer.train()
    # model.save_pretrained(os.path.join(config.output_dir, "model_multi_dataset"))
    # tokenizer.save_pretrained(os.path.join(config.output_dir, "tokenizer_multi_dataset"))

    # Evaluate multi-dataset
    # multi_dataset_metrics = evaluate_multi_dataset(model, tokenizer, individual_datasets)
    # results["multi_dataset"] = multi_dataset_metrics
    results["multi_dataset"] = "Training would execute here"

    print("\nTraining pipeline complete!")
    return results

# Define main function that executes the training pipeline
def main():
    """
    Main function to execute the full pipeline
    """
    print("LLM Finetuning with Chat Templates, Classification, Extended Context, and Multi-dataset Training")
    print("=" * 80)

    # Display configurations
    print(f"Base model: {config.model_name}")
    print(f"Default context size: {config.max_seq_length}")
    print(f"Extended context size: {config.extended_max_seq_length}")
    print(f"LoRA rank: {config.lora_r}")
    print(f"LoRA alpha: {config.lora_alpha}")
    print(f"Batch size: {config.batch_size}")
    print(f"Learning rate: {config.learning_rate}")
    print(f"Number of epochs: {config.num_epochs}")

    # Execute full pipeline
    results = full_training_pipeline()

    # Print summary of results
    print("\nSummary of Results:")
    for dataset_type, result in results.items():
        print(f"  - {dataset_type}: {result}")

    print("\nNote: Actual training was not executed in this example.")
    print("To run the actual training, uncomment the trainer.train() lines in the full_training_pipeline function.")

# Entry point
if __name__ == "__main__":
    main()


SECTION 6: COMBINED TRAINING PIPELINE
LLM Finetuning with Chat Templates, Classification, Extended Context, and Multi-dataset Training
Base model: TinyLlama/TinyLlama-1.1B-Chat-v1.0
Default context size: 512
Extended context size: 2048
LoRA rank: 8
LoRA alpha: 16
Batch size: 4
Learning rate: 0.0002
Number of epochs: 3
Starting full training pipeline...

Step 1: Training on conversational data...

Step 2: Training on classification data...

Step 3: Training extended context model...

Step 4: Training on multiple datasets...

Training pipeline complete!

Summary of Results:
  - conversational: Training would execute here
  - classification: Training would execute here
  - extended_context: Training would execute here
  - multi_dataset: Training would execute here

Note: Actual training was not executed in this example.
To run the actual training, uncomment the trainer.train() lines in the full_training_pipeline function.


In [23]:
from IPython.display import clear_output
clear_output()

SECTION 7: INFERENCE AND DEPLOYMENT

In [24]:
print("\n" + "="*50)
print("SECTION 7: INFERENCE AND DEPLOYMENT")
print("="*50)

def load_and_merge_model(model_path):
    """
    Load a trained LoRA model and merge weights with base model
    """
    # Load base model
    base_model = AutoModelForCausalLM.from_pretrained(
        config.model_name,
        torch_dtype=torch.float16,
        device_map="auto"
    )

    # Load and merge LoRA weights
    from peft import PeftModel

    # Load the LoRA model
    model = PeftModel.from_pretrained(
        base_model,
        model_path,
        device_map="auto"
    )

    # Merge LoRA weights with base model
    merged_model = model.merge_and_unload()

    return merged_model

def inference_demo(model, tokenizer, prompt, max_length=100):
    """
    Run inference on a model with a given prompt
    """
    # Format the prompt using chat template
    if isinstance(prompt, list):  # If prompt is a list of message dicts
        formatted_prompt = tokenizer.apply_chat_template(prompt, tokenize=False, add_generation_prompt=True)
    else:  # If prompt is a string
        formatted_prompt = prompt

    # Tokenize
    inputs = tokenizer(formatted_prompt, return_tensors="pt").to(model.device)

    # Generate
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_length,
        temperature=0.7,
        do_sample=True,
        top_p=0.9,
    )

    # Decode and return
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

def demo_all_models():
    """
    Demonstrate inference with all trained models
    """
    # Note: In a real scenario, you would load actual trained models from disk
    # For this example, we'll just show the code structure

    # List of model types and test prompts
    model_demos = [
        {
            "name": "Conversational",
            "path": os.path.join(config.output_dir, "model_conv"),
            "prompt": [
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": "What's your favorite book and why?"}
            ]
        },
        {
            "name": "Classification",
            "path": os.path.join(config.output_dir, "model_classification"),
            "prompt": [
                {"role": "system", "content": "You are a helpful assistant that classifies movie reviews as positive or negative."},
                {"role": "user", "content": "Classify the following review: 'This movie was absolutely fantastic! I loved the plot and the acting was superb.'"}
            ]
        },
        {
            "name": "Extended Context",
            "path": os.path.join(config.output_dir, "model_extended_context"),
            "prompt": [
                {"role": "system", "content": "You are a helpful assistant that summarizes long texts."},
                {"role": "user", "content": "Summarize the following text: " + "This is a test of extended context. " * 100}
            ]
        },
        {
            "name": "Multi-Dataset",
            "path": os.path.join(config.output_dir, "model_multi_dataset"),
            "prompt": [
                {"role": "system", "content": "You are a helpful coding assistant."},
                {"role": "user", "content": "Write a Python function to find the nth Fibonacci number."}
            ]
        }
    ]

    print("\nModel Inference Demos:")
    print("Note: This is showing code structure only, not actual inference since training wasn't executed.")

    for demo in model_demos:
        print(f"\n{'-'*40}")
        print(f"Model: {demo['name']}")
        print(f"Prompt: {demo['prompt'][-1]['content'][:50]}...")
        print(f"Response would be generated here if model was trained")
        # In a real scenario:
        # model = load_and_merge_model(demo["path"])
        # response = inference_demo(model, tokenizer, demo["prompt"])
        # print(f"Response: {response}")

# Deployment instructions
def print_deployment_guide():
    """
    Print instructions for deploying the finetuned models
    """
    print("\nDeployment Guide:")
    print("1. Export your model for production:")
    print("   ```python")
    print("   # Merge LoRA weights with base model")
    print("   merged_model = load_and_merge_model('path/to/finetuned/model')")
    print("   # Save the merged model")
    print("   merged_model.save_pretrained('path/to/export/model')")
    print("   tokenizer.save_pretrained('path/to/export/tokenizer')")
    print("   ```")

    print("\n2. Quantize for deployment (optional):")
    print("   ```python")
    print("   from transformers import BitsAndBytesConfig")
    print("   quantization_config = BitsAndBytesConfig(load_in_4bit=True)")
    print("   quantized_model = AutoModelForCausalLM.from_pretrained(")
    print("       'path/to/export/model',")
    print("       quantization_config=quantization_config,")
    print("       device_map='auto'")
    print("   )")
    print("   ```")

    print("\n3. Deploy using Hugging Face Inference Endpoints or other services")
    print("   - Push to Hugging Face Hub:")
    print("     ```python")
    print("     from huggingface_hub import HfApi")
    print("     api = HfApi()")
    print("     api.create_repo('your-username/your-model-name')")
    print("     merged_model.push_to_hub('your-username/your-model-name')")
    print("     tokenizer.push_to_hub('your-username/your-model-name')")
    print("     ```")

    print("\n4. For local deployment, use FastAPI:")
    print("   ```python")
    print("   from fastapi import FastAPI")
    print("   from pydantic import BaseModel")
    print("   import torch")
    print("   from transformers import AutoModelForCausalLM, AutoTokenizer")
    print("   ")
    print("   app = FastAPI()")
    print("   ")
    print("   # Load model and tokenizer")
    print("   model = AutoModelForCausalLM.from_pretrained('path/to/export/model', device_map='auto')")
    print("   tokenizer = AutoTokenizer.from_pretrained('path/to/export/tokenizer')")
    print("   ")
    print("   class ChatRequest(BaseModel):")
    print("       messages: list")
    print("   ")
    print("   @app.post('/generate')")
    print("   def generate_text(request: ChatRequest):")
    print("       formatted_prompt = tokenizer.apply_chat_template(request.messages, tokenize=False, add_generation_prompt=True)")
    print("       inputs = tokenizer(formatted_prompt, return_tensors='pt').to(model.device)")
    print("       outputs = model.generate(**inputs, max_new_tokens=100)")
    print("       response = tokenizer.decode(outputs[0], skip_special_tokens=True)")
    print("       return {'response': response.split(formatted_prompt)[-1]}")
    print("   ```")

# Call demo functions
print("\nInference examples:")
demo_all_models()
print_deployment_guide()



SECTION 7: INFERENCE AND DEPLOYMENT

Inference examples:

Model Inference Demos:
Note: This is showing code structure only, not actual inference since training wasn't executed.

----------------------------------------
Model: Conversational
Prompt: What's your favorite book and why?...
Response would be generated here if model was trained

----------------------------------------
Model: Classification
Prompt: Classify the following review: 'This movie was abs...
Response would be generated here if model was trained

----------------------------------------
Model: Extended Context
Prompt: Summarize the following text: This is a test of ex...
Response would be generated here if model was trained

----------------------------------------
Model: Multi-Dataset
Prompt: Write a Python function to find the nth Fibonacci ...
Response would be generated here if model was trained

Deployment Guide:
1. Export your model for production:
   ```python
   # Merge LoRA weights with base model
   merge

In [25]:
from IPython.display import clear_output
clear_output()

In [26]:
!nbstripout Phi-3.5ipynb